In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

In [ ]:
embeddings = OpenAIEmbeddings()
persist_directory = 'db_langchain_github'
db = Chroma(collection_name="langchain_github", persist_directory=persist_directory, embedding_function=embeddings)
db_retriever = db.as_retriever()

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain, LLMChain
from langchain.memory import ConversationBufferWindowMemory
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

In [ ]:
def generate_response(human_input):
    system_prompt_template = """You are expert programer. Use the provided data {context} to write a code and answer questions about provided code."""

    human_prompt_template = "{text}."
    system_message_prompt = SystemMessagePromptTemplate.from_template(system_prompt_template)
    human_message_prompt = HumanMessagePromptTemplate.from_template(human_prompt_template)
    # delete the gpt-4 model_name to use the default gpt-3.5 turbo for faster results
    gpt_4 = ChatOpenAI(temperature=.02)
    # Use the retriever's 'get_relevant_documents' method if needed to filter down longer docs
    relevant_nodes = db_retriever.get_relevant_documents(human_input)
    conversation = [system_message_prompt, human_message_prompt]
    chat_prompt = ChatPromptTemplate.from_messages(conversation)
    response = gpt_4(chat_prompt.format_prompt( 
        context=relevant_nodes, 
        text=human_input).to_messages())
    return response

In [ ]:
response = generate_response("write a python script to load text file to chroma data base")

In [ ]:
print(response.content)

In [ ]:
response = generate_response("write a python script to load text file to existing chroma data base")

In [ ]:
print(response.content)

In [ ]:
response = generate_response("write a python script to load web site from url to existing chroma data base")
print(response.content)

In [ ]:
response = generate_response("list available loaders")
print(response.content)

In [ ]:
response = generate_response("write a python script to load the whole web site from url to the local drive and then to existing chroma data base")
print(response.content)

In [ ]:
response = generate_response("use langchain to load html files from a folder to chromadb")
print(response.content)

In [ ]:
response = generate_response("explain vectorstore = Chroma(collection_name=langchain_github")
print(response.content)

In [ ]:
response = generate_response("db = Chroma(collection_name=\"langchain_github\", persist_directory=persist_directory, embedding_function=embeddings)db_retriever = db.as_retriever() How to retrieve information from several collection names")
print(response.content)

In [ ]:
response = generate_response("relevant_nodes = db_retriever.get_relevant_documents(human_input). How to use several relevant nodes?")
print(response.content)

In [ ]:
response = generate_response("relevant_nodes = db_retriever.get_relevant_documents(human_input). How to use several db_retriever?")
print(response.content)

In [ ]:
response = generate_response("response = gpt_4(chat_prompt.format_prompt( context=relevant_nodes,text=human_input).to_messages()). what are options for context?")
print(response.content)